In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
from os.path import isfile, join
from PIL import Image
import os.path, sys
import glob


## 1. specify folder

In [ ]:
path = "/Users/rubyjiang/Desktop/Keyhole-image-segmentation/application/model_output_masks/ML/ML_train_masks"
imgList=sorted(glob.glob(path+ '/*.tif'))
print(len(imgList))

## 2. specifiy features

In [ ]:
def extractFeatures(img):
    
    num_white_pixel = np.sum(img == 1) 
    num_white_pixel_row = np.sum(img == 1, 1) 
    # get the index of non zero rows
    nonzero_index = [i for i, e in enumerate(num_white_pixel_row) if e != 0]
    depth = len(nonzero_index)
    # avg width
    avg_width = 0
    if depth != 0:
        avg_width = np.sum(num_white_pixel_row) / depth
    width_top, width_quarter, width_half, width_3quarter = 0, 0, 0, 0
    # width
    if depth == 1:
        width_top = num_white_pixel_row[nonzero_index[0]] 
    if depth == 2:
        width_top = num_white_pixel_row[nonzero_index[0]] 
        width_quarter = num_white_pixel_row[nonzero_index[1]]
    if depth == 3:
        width_top = num_white_pixel_row[nonzero_index[0]] 
        width_quarter = num_white_pixel_row[nonzero_index[1]]
        width_half = num_white_pixel_row[nonzero_index[2]]
    if depth == 4:
        width_top = num_white_pixel_row[nonzero_index[0]] 
        width_quarter = num_white_pixel_row[nonzero_index[1]]
        width_half = num_white_pixel_row[nonzero_index[2]]
        width_3quarter = num_white_pixel_row[nonzero_index[3]]
    if depth >= 5:
        # index
        top_index = nonzero_index[0]
        quarter_index = top_index + round(len(nonzero_index) * 0.25)
        half_index = top_index + len(nonzero_index)//2 
        threequarter_index = top_index + round(len(nonzero_index) * 0.75)
        #
        width_top = num_white_pixel_row[top_index]
        width_quarter = num_white_pixel_row[quarter_index]
        width_half = num_white_pixel_row[half_index]
        width_3quarter = num_white_pixel_row[threequarter_index]
    # widest width
    wwidth = 0
    if depth !=0:
        min_col = len(img)
        max_col = 0
        for row in range(len(img)):
            for col in range(len(img[0])):
                if img[row][col] != 0 :
                    min_col = min(min_col, col)
                    max_col = max(max_col, col)
        wwidth = max(wwidth, max_col - min_col)   
    # perimeter
    perimeter = 0
    contours,hierarchy = cv.findContours(img, 1, 2)
    if contours:
        cnt = contours[0]
        perimeter = cv.arcLength(cnt,True)
    # front wall angle 
    tan_theta = None
    if depth >= 5:
        up_index =  nonzero_index[round(len(nonzero_index)/10)]
        down_index =  nonzero_index[round(len(nonzero_index)*9/10)]
        right_index = np.nonzero(img[up_index])[-1][-1]
        left_index = np.nonzero(img[down_index])[-1][-1]
        if abs(right_index-left_index) == 0:
            tan_theta = 180 # just to make sure theta is 90
        else:
            tan_theta = abs(up_index-down_index)/abs(right_index-left_index)

        
    return [depth,
            width_top,
            width_quarter,
            width_half,
            width_3quarter,
            avg_width,
            wwidth,
            num_white_pixel,
            perimeter,
            tan_theta,
           ]

In [ ]:
data= {
    'img_name':[],
    'depth':[],
    'width_top' :[],
    'width_quarter':[],
    'width_half':[],
    'width_3quarter':[],
    'avg_width':[],
    'widest_width':[],
    'num_white_pixel':[],
    'perimeter':[],
    'tan_theta': []
          }
df = pd.DataFrame(data)
print(len(df))

In [ ]:
for i in range(len(imgList)):
    frame = imgList[i]
    name = frame.split('/')[-1]
    print(name)
    # img
    img = cv.imread(frame, cv.IMREAD_UNCHANGED).astype(np.uint8)
    #cv.imshow('Original Image', img)
    # rotate colockwise 7 degree
    height, width = img.shape[:2]
    # Define rotation matrix using cv2.getRotationMatrix2D
    rotation_matrix = cv.getRotationMatrix2D((width/2, height/2), -7, 1)
    # Apply rotation using cv2.warpAffine
    img = cv.warpAffine(img, rotation_matrix, (width, height))
    # Flip image horizontally using cv2.flip
    img = cv.flip(img, 1)
    
    # contour
    contours,hierarchy = cv.findContours(img, 1, 2)
    areas = [cv.contourArea(contour) for contour in contours]
    if len(areas) != 0:
        keyhole_index = np.argmax(areas)
#         if i != 836-1: # special for frame 836 in train_masks
#             keyhole_index = np.argmax(areas)
#         else:
#             keyhole_index = np.argmin(areas)
        pores_index = [i for i in range(len(areas)) if i != keyhole_index]
        img = cv.fillPoly(img,pts= [contours[i] for i in pores_index],color = 0)
    feature_list = extractFeatures(img)
    #print(feature_list)
    df.loc[i] = [name] + feature_list

In [ ]:
len(df)

In [ ]:
df[]

In [ ]:
plt.plot(df['depth'][430:460])

In [ ]:
df.describe()

In [ ]:
save_path = '/Users/rubyjiang/Desktop/'
df.to_csv(save_path + 'ML_train_features.csv', index = False, header=True) #,float_format='%.8f')